# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, desc, asc, explode, lit, min, max, split, udf, lit, when, lower
from pyspark.sql.functions import to_timestamp, year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import IntegerType, StringType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re
import datetime

In [2]:
# create a Spark session

spark = SparkSession.builder \
    .appName("Sparkify") \
    .getOrCreate()

In [3]:
spark

In [17]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [5]:
df = spark.read.json('../data/mini_sparkify_event_data.json')

In [6]:
df.head(2)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9')]

In [7]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
df.describe().show(1,vertical=True)

-RECORD 0---------------
 summary       | count  
 artist        | 228108 
 auth          | 286500 
 firstName     | 278154 
 gender        | 278154 
 itemInSession | 286500 
 lastName      | 278154 
 length        | 228108 
 level         | 286500 
 location      | 278154 
 method        | 286500 
 page          | 286500 
 registration  | 278154 
 sessionId     | 286500 
 song          | 228108 
 status        | 286500 
 ts            | 286500 
 userAgent     | 278154 
 userId        | 286500 
only showing top 1 row



In [10]:
df.distinct().count()

286500

In [13]:
def clean_df(spark, df=df):
    '''
    cleaning database
    : INPUT
    spark - spark session
    df - dataframe
    :OUTPUT
    df- cleaned Dataframe
    '''
    
    #filter empty values in userId
    df=df.filter(df.userId != '')
    
    # create new columns with iso timestamp using ts and registration columns
    df=df.withColumn('event_timestamp', to_timestamp(df['ts']/1000))
    df=df.withColumn('registration_timestamp', to_timestamp(df['registration']/1000))
    
    # create new columns for device and device_os using userAgent column
    df=df.withColumn('device', 
                      when(lower(df.userAgent).contains('windows'), lit('computer'))
                      .when(lower(df.userAgent).contains('macintosh'), lit('computer'))
                      .when(lower(df.userAgent).contains('linux'), lit('computer'))
                      .when(lower(df.userAgent).contains('mobile'), lit('mobile'))
                      .otherwise(lit('other')) )

    df=df.withColumn('device_os', 
                      when(lower(df.userAgent).contains('windows'), lit('windows'))
                      .when(lower(df.userAgent).contains('mac'), lit('mac'))
                      .when(lower(df.userAgent).contains('linux'), lit('linux'))
                      .otherwise(lit('other')) )
    
    #split lacation into seperate city and state columns
    df=df.withColumn('city', split(df['location'],',').getItem(0))
    df=df.withColumn('state', split(df['location'],',').getItem(1))
    
    # create churn and downgrade columns from page columns
    df=df.withColumn('churn', 
                      when(df.page.contains('Cancellation Confirmation'), lit(1))
                      .otherwise(lit(0)) )
    df=df.withColumn('downgrade', 
                      when(df.page.contains('Downgrade'), lit(1))
                      .otherwise(lit(0)) )
    
    return df

df=clean_df(spark, df)

In [16]:
df.show(20)

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+-------------------+----------------------+--------+---------+--------------------+------+-----+---------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|    event_timestamp|registration_timestamp|  device|device_os|                city| state|churn|downgrade|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+-------------------+----------------------+--------+---------+--------------------+------+-----+---------+
|

In [18]:
df.createOrReplaceTempView("events")
user_table = spark.sql("""
                        SELECT 
                            u.userId as user_id, 
                            u.firstName as first_name, 
                            u.lastName as last_name, 
                            u.gender as gender, 
                            u.location as location,
                            u.city as city,
                            u.state as state,
                            u.userAgent as user_agent,
                            u.device as device,
                            u.device_os as device_os,
                            u.registration as registration,
                            u.registration_timestamp as registration_timestamp,
                            p.churn as churn,
                            p.downgrade as downgrade
                        FROM (SELECT DISTINCT
                                userId , 
                                firstName, 
                                lastName, 
                                gender, 
                                location,
                                city,
                                state,
                                userAgent,
                                device,
                                device_os ,
                                registration ,
                                registration_timestamp FROM events) AS u
                        JOIN (SELECT DISTINCT
                                userId, 
                                MAX(churn) as churn,
                                MAX(downgrade) as downgrade
                                FROM events GROUP BY userId) AS p
                        ON u.userId=p.userId
                        """)

user_table.show(5)

+-------+----------+---------+------+--------------------+--------------------+-----+--------------------+--------+---------+-------------+----------------------+-----+---------+
|user_id|first_name|last_name|gender|            location|                city|state|          user_agent|  device|device_os| registration|registration_timestamp|churn|downgrade|
+-------+----------+---------+------+--------------------+--------------------+-----+--------------------+--------+---------+-------------+----------------------+-----+---------+
|     51|     Ethan|  Johnson|     M|Lexington-Fayette...|   Lexington-Fayette|   KY|"Mozilla/5.0 (Win...|computer|  windows|1538080987000|   2018-09-27 22:43:07|    1|        1|
|    124|    Nicole|     Beck|     F|Vineland-Bridgeto...|  Vineland-Bridgeton|   NJ|"Mozilla/5.0 (Mac...|computer|      mac|1532224335000|   2018-07-22 03:52:15|    0|        1|
|      7|    Maddox|  Johnson|     M|Seattle-Tacoma-Be...|Seattle-Tacoma-Be...|   WA|Mozilla/5.0 (Wind...

In [ ]:
df.createOrReplaceTempView("events")
event_table = spark.sql("""
                    SELECT DISTINCT
                        userId as user_id,
                        sessionId as session_id,
                        itemInSession as item_in_session,
                        page as event,
                        auth,
                        level,
                        ts as event_timestamp_unix,
                        event_timestamp                   
                    FROM events
                    ORDER BY event_timestamp_unix DESC
                """) 
event_table.show(10)

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.

In [ ]:
#spark.stop()